# Min-max algorithm과 alpha-beta pruning을 활용한 tic-tac-toe agent

In [31]:
import numpy as np

In [32]:
GOING_FIRST = 1
GOING_SECOND = -1
EMPTY_TILE = 0

RENDERING_SYMBOL = {GOING_FIRST : 'O', GOING_SECOND : 'X', EMPTY_TILE : '-'}

BOARD_ROWS = 3
BOARD_COLS = 3

In [33]:
'''
Action Index for each position

(0,0) : 0, (0,1) : 1, (0,2) : 2,
(1,0) : 3, (1,1) : 4, (1,2) : 5,
(2,0) : 6, (2,1) : 7, (2,2) : 8
'''
def pos_to_actidx(row_idx, col_idx):
    action_idx = -1
    if 0 <= row_idx < BOARD_ROWS and 0 <= col_idx < BOARD_COLS:
        action_idx = 3*row_idx + col_idx
    return action_idx

In [34]:
class State:
    def __init__(self, board_rows=BOARD_ROWS, board_cols=BOARD_COLS):
        self.board_rows = board_rows
        self.board_cols = board_cols
        self.board_size = board_rows * board_cols

        self.board = np.zeros(shape=[board_rows, board_cols], dtype=int) # 3X3 게임판 초기 상태

        self.empty_tile = EMPTY_TILE # 0
        self.going_first = GOING_FIRST # 1
        self.going_second = GOING_SECOND # -1

        self.rendering_symbol = RENDERING_SYMBOL

        self.winner = 0
        self.end = False

    # 가능한 액션 인덱스를 반환하는 함수 (수를 둘 수 있는 위치가 어딘지 알려주는 함수)
    def get_available_actions(self):
        if self.is_end_state():
            return []

        available_positions = np.argwhere(self.board == self.empty_tile)

        available_action_ids = [pos_to_actidx(pos[0], pos[1]) for pos in available_positions]

        return available_action_ids

    # 게임 종료 여부
    def is_end_state(self):
        if self.end:
            return self.end

        row_sums = np.sum(self.board, axis=1)  # 각 행의 합
        col_sums = np.sum(self.board, axis=0)  # 각 열의 합

        # 대각선의 합
        trace = np.trace(self.board)  # 왼쪽 위 ~ 오른쪽 아래로
        reverse_trace = np.trace(np.fliplr(self.board))  # 오른쪽 위 ~ 왼쪽 아래
        ## np.fliplr()
        ## [[1 2 3]  --->  [[3 2 1]
        ##  [4 5 6]         [6 5 4]
        ##  [7 8 9]]        [9 8 7]]

        results = np.concatenate((row_sums, col_sums, [trace, reverse_trace]))

        # GOING_FIRST(선공) 또는 GOING_SECOND(후공) 승리 조건 확인
        for result in results:
            if result == 3 or result == -3:
                self.end = True
                if result == 3:
                    self.winner = self.going_first
                else:
                    self.winner = self.going_second
                return self.end

        # 무승부 확인: 모든 칸이 채워졌는지 확인 (보드의 절댓값의 합이 총 칸 수와 같다면 무승부)
        if np.sum(np.abs(self.board)) == self.board_size:
            self.winner = 0
            self.end = True
            return self.end

        # 게임이 아직 끝나지 않음
        self.end = False
        return self.end

    def get_winner(self):
        if self.winner == self.going_first:
            return 1
        elif self.winner == self.going_second:
            return -1
        else:
            return 0

    # render 함수
    def get_state_as_board(self):
        int_to_symbol = np.where(self.board == 1, 'O',
                                np.where(self.board == -1, 'X', '-'))

        rendering_board = '\n'.join([' '.join(row) for row in int_to_symbol])

        return rendering_board

In [35]:
class TicTacToeEnv:
    def __init__(self, agent1, agent2):
        self.state = State()
        self.agent1 = agent1
        self.agent2 = agent2
        self.current_turn = self.state.going_first  # 선공부터 시작

        self.action_space = np.arange(self.state.board_size)
        self.num_actions = len(self.action_space)

        self.reward_dict = {'win': 10, 'lose': -10, 'draw': 0, 'continue': 0}

    def reset(self):
        self.state = State()
        self.current_turn = self.state.going_first
        return self.state

    def step(self):
        if self.current_turn == self.state.going_first:
            action = self.agent1.get_action(self.state)
            agent_role = self.state.going_first
            agent = self.agent1
        else:
            action = self.agent2.get_action(self.state)
            agent_role = self.state.going_second
            agent = self.agent2

        # 행동 적용
        x, y = np.divmod(action, self.state.board_cols)
        if self.state.board[x, y] != self.state.empty_tile:
            raise ValueError(f"Invalid action: Tile ({x}, {y}) is already occupied.")

        self.state.board[x, y] = agent_role

        # 상태 업데이트
        self.state.end = self.state.is_end_state()

        # 턴 교대
        self.current_turn = self.state.going_second if self.current_turn == self.state.going_first else self.state.going_first

        # 보상 및 종료 여부 반환
        if self.state.end:
            if self.state.winner == self.state.going_first:
                reward = self.reward_dict['win'] if agent_role == self.state.going_first else self.reward_dict['lose']
                done = True
                result = 'agent1 wins' if agent_role == self.state.going_first else 'agent2 wins'
            elif self.state.winner == self.state.going_second:
                reward = self.reward_dict['win'] if agent_role == self.state.going_second else self.reward_dict['lose']
                done = True
                result = 'agent2 wins' if agent_role == self.state.going_second else 'agent1 wins'
            else:
                reward = self.reward_dict['draw']
                done = True
                result = 'draw'
        else:
            reward = self.reward_dict['continue']
            done = False
            result = 'continue'

        return self.state, reward, done, result

    def render(self):
        return self.state.get_state_as_board()


In [36]:
class MinMaxAgent:
    def __init__(self, role):
        self.role = role  # 1 for 선공, -1 for 후공

    def get_action(self, state: State):
        depth = self.get_current_depth(state)  # 현재 상태에서 가능한 최대 탐색 깊이
        action, _ = self.minmax(state, depth, True)  # MAX (agent)의 차례
        return action  # 최적의 행동 반환

    def get_current_depth(self, state: State):
        empty_tiles = np.sum(state.board == state.empty_tile)  # 빈 칸 개수
        return empty_tiles  # 남은 빈 칸의 개수가 탐색할 수 있는 depth

    def minmax(self, state: State, depth, maxPlayer: bool):
        action = -1

        # Terminal node이면 보드를 평가해 위치와 평가값을 반환
        if depth == 0 or state.is_end_state():
            return action, self.evaluate(state)

        if maxPlayer:
            value = -np.inf
            for a in state.get_available_actions():  # a : action index
                new_state = self.apply_action(state, a, state.going_first)
                _, score = self.minmax(new_state, depth - 1, False)  # 다음 턴; 후공 MIN 차례
                if score > value:  # score가 현재 가지고 있는 value보다 크면
                    value = score  # 최댓값을 취한다.
                    action = a  # (최댓값을 얻을 수 있는 상태로 갈 수 있는) 액션 인덱스
        else:
            value = np.inf
            for a in state.get_available_actions():
                new_state = self.apply_action(state, a, state.going_second)
                _, score = self.minmax(new_state, depth - 1, True)
                if score < value:
                    value = score  # 최솟값을 취한다.
                    action = a

        return action, value

    def apply_action(self, state: State, action: int, player: int):
        new_state = State()
        new_state.board = state.board.copy()
        new_state.board[action // state.board_cols, action % state.board_cols] = player
        new_state.is_end_state()  # 업데이트된 상태의 종료 여부를 확인
        new_state.winner = state.get_winner()
        return new_state

    def evaluate(self, state: State):
        winner = state.get_winner()

        if self.role == state.going_first:  # MAX 플레이어가 선공인 경우
            if winner == state.going_first:  # MAX 승리
                return 1
            elif winner == state.going_second:  # 상대방 승리
                return -1
            else:
                return 0  # 무승부 또는 진행 중인 상태

        elif self.role == state.going_second:  # 후공인 경우
            if winner == state.going_second:
                return 1
            elif winner == state.going_first:
                return -1
            else:
                return 0


In [37]:
class AlphaBetaAgent:
    def __init__(self, role):
        self.role = role  # 1 for 선공, -1 for 후공

    def get_action(self, state: State):
        depth = self.get_current_depth(state)  # 현재 상태에서 가능한 최대 탐색 깊이
        action, _ = self.alphabeta(state, depth, -np.inf, np.inf, True)  # MAX (agent)의 차례
        return action  # 최적의 행동 반환

    def get_current_depth(self, state: State):
        empty_tiles = np.sum(state.board == state.empty_tile)  # 빈 칸 개수
        return empty_tiles  # 남은 빈 칸의 개수가 탐색할 수 있는 depth

    def alphabeta(self, state: State, depth, alpha, beta, maxPlayer: bool):
        action = -1

        if depth == 0 or state.is_end_state():
            return action, self.evaluate(state)

        if maxPlayer:
            value = -np.inf
            for a in state.get_available_actions():
                new_state = self.apply_action(state, a, state.going_first)
                _, score = self.alphabeta(new_state, depth - 1, alpha, beta, False)
                if score > value:
                    value = score
                    action = a
                alpha = max(alpha, value)
                if alpha >= beta:
                    break  # beta 컷
        else:
            value = np.inf
            for a in state.get_available_actions():
                new_state = self.apply_action(state, a, state.going_second)
                _, score = self.alphabeta(new_state, depth - 1, alpha, beta, True)
                if score < value:
                    value = score
                    action = a
                beta = min(beta, value)
                if alpha >= beta:
                    break  # alpha 컷

        return action, value

    def apply_action(self, state: State, action: int, player: int):
        new_state = State()
        new_state.board = state.board.copy()
        new_state.board[action // state.board_cols, action % state.board_cols] = player
        new_state.is_end_state()  # 업데이트된 상태의 종료 여부를 확인
        new_state.winner = state.get_winner()
        return new_state

    def evaluate(self, state: State):
        winner = state.get_winner()

        if self.role == state.going_first:  # MAX 플레이어가 선공인 경우
            if winner == state.going_first:  # MAX 승리
                return 1
            elif winner == state.going_second:  # 상대방 승리
                return -1
            else:
                return 0  # 무승부 또는 진행 중인 상태

        elif self.role == state.going_second:  # 후공인 경우
            if winner == state.going_second:
                return 1
            elif winner == state.going_first:
                return -1
            else:
                return 0

In [43]:
def play_game(env: TicTacToeEnv, render: bool = True):
    env.reset()
    while True:
        state, reward, done, result = env.step()
        if render:
            print(state.get_state_as_board())
            print(result)
            print("-" * 10)
        if done:
            break


agent1 = AlphaBetaAgent(role=GOING_FIRST)
agent2 = MinMaxAgent(role=GOING_SECOND)

env = TicTacToeEnv(agent1, agent2)

num_games = 1
for i in range(num_games):
    print(f"Game {i+1}")
    play_game(env, render=True)

    # 역할 교대
    env.reset()
    env.agent1, env.agent2 = env.agent2, env.agent1
    env.agent1.role, env.agent2.role = env.agent2.role, env.agent1.role

    print("-" * 10)

Game 1
O - -
- - -
- - -
continue
----------
O X -
- - -
- - -
continue
----------
O X O
- - -
- - -
continue
----------
O X O
X - -
- - -
continue
----------
O X O
X O -
- - -
continue
----------
O X O
X O X
- - -
continue
----------
O X O
X O X
O - -
agent1 wins
----------
----------


# Memo

\<Tic-Tac-Toe 규칙>

- 선공, 후공 차례대로 번갈아가면서 수를 둔다.

- Max가 선공. (선공 : 'O', +1)

- 같은 무늬의 직선 한 줄(가로, 세로, 대각)을 먼저 완성하는 쪽이 승리.

제로썸 게임이다. (승자와 패자가 나뉜다.)

MinMax 알고리즘을 사용하기에 플레이어들은 최선의 수를 선택한다는 가정하에 게임을 진행한다.

***
***

Zero-sum game

: 어떤 한 플레이어에게 좋은 건 반대로 다른 플레이어에게는 나쁜 게임을 의미한다. 자신이 점수를 얻으면 상대방은 결국 점수를 잃는 것이고, 두 사람의 결과를 더해보면 0이 되는 것이다.

***
***

### Min-max algorithm

: 턴제 게임에서 상대방이 최선의 수를 택할 것이라고 가정하고 다음 수를 예측한다. **상대방의 최선의 수는 나의 입장에서는 최악의 수**이기 때문에 '상대방의 최선의 수를 두더라도 이에 대응하여 최소한의 손실을 보는 것'이 궁극적인 목표라 할 수 있다.

(선공자 입장에서의 알고리즘) 선공: 내 최선의 수(max) → 후공: 상대방 최선의 수(내 기준 최악의 수)(min) → ...

상대방인 MIN 플레이어가 나를 지게 만들기 위해 최선의 선택을 하는 worst-case에서 MAX 플레이어인 내 결과를 최대화하는 행동을 택한다.

두 플레이어(최대화 플레이어와 최소화 플레이어)가 번갈아 가며 최적의 수를 찾기 위해 사용됩니다. 각 단계에서:

최대화 플레이어는 자신의 점수를 최대화하려고 하고,
최소화 플레이어는 상대방(최대화 플레이어)의 점수를 최소화하려고 합니다.

**단말 노드의 평가 함숫값이 높을수록 MAX가 유리하고, 낮을수록 MIN이 유리하다.**

MAX 플레이어는 가능한 모든 행동(착수 위치)을 시뮬레이션하면서, 그 행동을 할 경우 MIN 플레이어가 취할 수 있는 최솟값을 추정합니다. 이는 `minimax(state, depth-1, False)`로 구현됩니다. 즉, **MAX는 MIN이 자신에게 최악의 경우를 만들려고 할 때, 그 결과를 분석**하는 겁니다.

score > value 조건에서, MAX 플레이어는 항상 더 높은 점수를 선택하려고 합니다. 이때 value는 MAX가 고려 중인 최댓값입니다. **score는 MIN의 수를 고려한 후의 예상 점수인데, 이 값이 현재 MAX가 갖고 있는 최댓값보다 크면, 더 나은 결과이므로 그 값을 업데이트합니다.**
***
***

### Alpha-beta pruning

: 최선의 수를 두는 플레이어의 선택에 최종적으로 영향을 끼치지 않는 가지를 삭제

Alpha : 현재까지 내가 이기기 위해 찾은 최고 점수

Beta : 현재까지 상대방이 나를 지게 만들기 위해 찾은 최저 점수

알파베타 가지치기의 핵심은 각 플레이어가 자신에게 유리한 선택을 할 때, 상대가 더 나은 선택을 방해할 수 있는지 미리 확인하여 탐색을 줄이는 것!!

※ **α >= β일 때 가지치기가 발생하는 이유**

- 알파(α)와 베타(β)의 의미

    α (알파): MAX 플레이어가 현재까지 찾은 최고의 점수. MAX는 가능한 한 이 알파보다 높은 점수를 얻으려고 한다.

    β (베타): MIN 플레이어가 현재까지 찾은 최악의 점수. MIN은 가능한 한 이 베타보다 낮은 점수를 얻으려고 한다.
    ***

- MAX와 MIN의 전략 차이

    MAX 플레이어는 가장 높은 점수를 얻으려고 하고,

    MIN 플레이어는 가장 낮은 점수를 얻으려고 한다.
    ***

- 핵심 아이디어: 왜 α >= β일 때 가지치기할까?

    α >= β라는 상태는, MAX와 MIN의 목표가 충돌하여 더 이상 탐색할 필요가 없다는 걸 의미함

    → 단계별 이해

    1. 현재까지 확인된 알파 값 α:

    MAX는 현재까지 탐색한 노드들에서 자신에게 유리한 값을 찾아가는 중. 이때 최고로 유리한 값이 알파(α)이다! 즉, **MAX는 이 알파보다 낮은 점수는 선택하지 않을 것**.

    2. 현재까지 확인된 베타 값 β:

    MIN은 상대방(MAX)이 탐색하는 도중, 자신이 막을 수 있는 최선의 방법을 찾고 있습니다. MIN이 찾은 가장 좋은 선택(**자신에게 최악인 점수 중에 최소의 값**)이 베타(β). **MIN은 이 베타보다 높은 점수는 주지 않으려고 할 것이다.**

    3. 알파, 베타의 의미가 충돌하는 순간 (α >= β):

    MAX는 현재 α 이상의 점수를 이미 확보할 수 있는 상황. 즉, **MAX는 α보다 낮은 점수는 고려하지 않을 것이고, 그 이상을 노리려고 함.**

    MIN은 현재 β 이하의 점수로 막으려고 하고 있다. 그런데 이 상황에서 α가 β보다 크거나 같은 경우, MIN은 더 낮은 점수를 선택할 필요가 없다.**어차피 MAX는 MIN의 베타 값보다도 더 큰 알파 값 이상의 값을 확보했기 때문에!**

# Reference

틱택토 State

\<레퍼런스>

[rl_practice_2_5.ipynb](https://colab.research.google.com/drive/1pMYjlOL2JUjlOovl6mlEkipf4SQOjkcO?usp=sharing)

Min-max algorithm, alpha-beta pruning

\<레퍼런스>

[위키백과 - 알파-베타 가지치기](https://ko.wikipedia.org/wiki/%EC%95%8C%ED%8C%8C-%EB%B2%A0%ED%83%80_%EA%B0%80%EC%A7%80%EC%B9%98%EA%B8%B0#cite_note-RN10-1)

[[인공지능 기초] Adversarial Search - Minimax Search와 Alpha-beta Pruning](https://glanceyes.com/entry/Adversarial-Search%EC%9D%98-Minimax-Search%EC%99%80-Alpha-beta-Pruning)

[MinMax알고리즘을 이용한 TicTacToe](https://velog.io/@h_ani99/MinMax%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-TicTacToe)

[알파-베타 가지치기, Alpha-beta pruning](https://going-to-end.tistory.com/entry/%EC%95%8C%ED%8C%8C-%EB%B2%A0%ED%83%80-%EA%B0%80%EC%A7%80%EC%B9%98%EA%B8%B0-Alpha-beta-pruning)